## Kelley hu4d5-5 VH-A78L error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VH-A78L TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [3]:
import os
os.chdir("..")
from common_functions import *

### Ingesting initial TI data 

In [4]:
os.chdir("./TI_data/VH-A78L/")
sc_dvdls = pd.read_csv("A78L_bound.csv")
dvdls_ub = pd.read_csv("A78L_unbound.csv")

### Original ddG estimate:

In [5]:
orig_dG_bd = sc_dvdls.groupby("Lambda").mean().sum()["weight_dvdl"]
orig_dG_ubd = dvdls_ub.groupby("Lambda").mean().sum()["weight_dvdl"]

empirical_value = 0.00
orig_error = abs((orig_dG_bd - orig_dG_ubd) - empirical_value)

print("Original ddG estimate: ")
print(f"{round(orig_dG_bd - orig_dG_ubd, 5)} kcal/mol ")
print()
print("Original ddG error: ")
print(f"{round(orig_error, 5)} kcal/mol")

Original ddG estimate: 
0.43097 kcal/mol 

Original ddG error: 
0.43097 kcal/mol


### Removing correlated variables among candidate degrees of freedom

This is important because we want to limit the noise in our model training. See our methods/supplemental methods section for our process to choose the input features.

In [6]:
X = sc_dvdls.drop([
    "#Frame", "weight_dvdl", "dvdl", "Run", "Lambda", "N426_chi2"
], axis=1)

Y = sc_dvdls["weight_dvdl"]

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns


### Checking to see if there is any cross-correlations within the independent variables

In [7]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. 

In [12]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, sc_dvdls["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.3176
Training r2 std dev: 
0.0008
Avg. test r2: 
0.1042
Testing r2 std dev: 
0.0063


,Mean,Median
L570_chi2,0.147999,0.148628
L570_chi1,0.089902,0.089332
N426_chi1,0.068813,0.074732
W385_chi2,0.067422,0.075003
I419_chi1,0.059866,0.065065
T427_chi1,0.052580,0.035703
D422_chi2,0.052436,0.057263
S370_chi1,0.050409,0.050889
I378_chi2,0.048896,0.054042
W385_chi1,0.047666,0.037609
